# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup.
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kingston,17.9970,-76.7936,28.53,65,75,1.03,JM,1734911636
1,1,fort mcmurray,56.7268,-111.3810,-7.23,92,100,4.12,CA,1734911518
2,2,sovetskaya,44.7769,41.1722,3.82,75,71,2.14,RU,1734911794
3,3,hohhot,40.8106,111.6522,-18.84,78,0,3.00,CN,1734911795
4,4,san antonio de pale,-1.4014,5.6325,26.42,81,86,4.20,GQ,1734911796


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    x="Lng",                     # Longitude for x-axis
    y="Lat",                     # Latitude for y-axis
    size="Humidity",             # Point size based on Humidity
    color="Humidity",            # Point color based on Humidity
    geo=True,                    # Enable geographic plotting
    tiles="CartoLight",          # Use a light map background
    frame_width=800,             # Width of the plot
    frame_height=600,            # Height of the plot
    title="City Humidity Map"    # Title of the map
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df[
    (city_data_df["Max Temp"] >= 2) &
    (city_data_df["Max Temp"] <= 90) &
    (city_data_df["Humidity"] < 90) &
    (city_data_df["Cloudiness"] < 90)
]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities.head()
ideal_cities.shape

(208, 10)

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()



,City,Country,Lat,Lng,Humidity,Hotel Name
0,kingston,JM,17.9970,-76.7936,65,
2,sovetskaya,RU,44.7769,41.1722,75,
4,san antonio de pale,GQ,-1.4014,5.6325,81,
7,timimoun,DZ,29.2639,0.2310,46,
12,san juan de miraflores,PE,-12.1629,-76.9635,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
# Set search radius (10,000 meters)
radius = 10000

# Base URL for Geoapify API
base_url = "https://api.geoapify.com/v2/places"

# Define parameters for the API search
params = {
    "categories": "accommodation.hotel",  # Looking for hotels
    "apiKey": geoapify_key,  # Your Geoapify API key
    "limit": 1  # Limit to the first result
}

In [7]:
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    print (lat) 
    print (lng) 
   # Add latitude and longitude to the filter parameter
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Make an API request
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    print(name_address)

    # Grab the first hotel from the results and store the name in the DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

Starting hotel search
17.997
-76.7936
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Aphrodite Hotel', 'country': 'Jamaica', 'country_code': 'jm', 'county': 'Saint Andrew', 'state_district': 'Surrey County', 'city': 'Kingston', 'district': 'Collins Green', 'suburb': 'Cross Roads', 'street': 'Beechwood Avenue', 'housenumber': '37', 'lon': -76.79791922357445, 'lat': 17.996573650000002, 'formatted': 'Aphrodite Hotel, 37 Beechwood Avenue, Cross Roads, Kingston, Jamaica', 'address_line1': 'Aphrodite Hotel', 'address_line2': '37 Beechwood Avenue, Cross Roads, Kingston, Jamaica', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Aphrodite Hotel', 'osm_id': 375996124, 'tourism': 'hotel', 'building': 'hot

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kingston,JM,17.9970,-76.7936,65,Aphrodite Hotel
2,sovetskaya,RU,44.7769,41.1722,75,No hotel found
4,san antonio de pale,GQ,-1.4014,5.6325,81,No hotel found
7,timimoun,DZ,29.2639,0.2310,46,أناف
12,san juan de miraflores,PE,-12.1629,-76.9635,68,Venue Hospedaje


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
import hvplot.pandas

# Add hover information to the DataFrame
hotel_df["Hover Info"] = (
    "City: " + hotel_df["City"] +
    "Country: " + hotel_df["Country"] +
    "Hotel: " + hotel_df["Hotel Name"] +
    "Humidity: " + hotel_df["Humidity"].astype(str) + "%"
)

# Create a map with the hover information
city_map = hotel_df.hvplot.points(
    "Lng", "Lat",  # Longitude and Latitude for the map
    geo=True,  # Enable geographic plotting
    tiles="CartoLight",  # Map background
    color="Humidity",  # Color based on Humidity
    size=10,  # Size of the points
    hover_cols=["Hover Info"],  # Add the hover information
    title="City Map with Hotel Information"
)

# Display the map
city_map


